In [12]:
import pandas as pd
import numpy as np
import joblib as jb
from scipy.stats import spearmanr
%matplotlib inline

In [30]:
train_train_level0 = jb.load("train_train_level0.pkl.z")
train_train_level1 = jb.load("train_train_level1.pkl.z")
train_valid_level1 = jb.load("train_valid_level1.pkl.z")
test = pd.read_feather("test.f")


X_train0, y_train0 = train_train_level0.filter(regex=r'feature', axis=1), train_train_level0['target_kazutsugi']
X_train1, y_train1 = train_train_level1.filter(regex=r'feature', axis=1), train_train_level1['target_kazutsugi']
X_val1, y_val1 = train_valid_level1.filter(regex=r'feature', axis=1), train_valid_level1['target_kazutsugi']
X_test = test.filter(regex=r'feature', axis=1)

# Classifier

In [14]:
y_train0.unique()

array([0.75, 0.25, 0.  , 0.5 , 1.  ])

In [15]:
labels = {0.: 0, 0.25: 1, 0.5: 2, 0.75: 3, 1.:4}
y_train0 = y_train0.map(labels)
y_train0

0         3
1         1
2         0
3         0
4         3
         ..
142092    0
142093    2
142094    1
142095    1
142096    0
Name: target_kazutsugi, Length: 142097, dtype: int64

In [16]:
label_values = np.array([0., 0.25, 0.5, 0.75, 1.0])

In [24]:
from xgboost import XGBClassifier


mdl = XGBClassifier(max_depth=5, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1, tree_method='gpu_hist', gpu_id=0)
mdl.fit(X_train0, y_train0)

p = mdl.predict_proba(X_train1)
p = p.dot(label_values)

model_name_train1 = "./preds_train1/xgb_classifier.pkl.z"
jb.dump(p, model_name_train1)

metric = spearmanr(y_train1.values,p).correlation

p = mdl.predict_proba(X_val1)
p = p.dot(label_values)

model_name_val1 = "./preds_val1/xgb_classifier.pkl.z"
jb.dump(p, model_name_val1)

p = mdl.predict_proba(X_test)
p = p.dot(label_values)

model_name_test = "./preds_test/xgb_classifier.pkl.z"
jb.dump(p, model_name_test)

print(metric)

# poderia usar as próprias probs na stack tb

0.037787336613260995


# Ranker

In [40]:
groups = train_train_level0.groupby("era").size().values
X_train0, y_train0 = train_train_level0.filter(regex=r'feature', axis=1), train_train_level0['target_kazutsugi']

In [41]:
%%time
from xgboost import XGBRanker


mdl = XGBRanker(max_depth=5, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1, tree_method='gpu_hist', gpu_id=0)
mdl.fit(X_train0, y_train0, group=groups)

p_tr1 = np.zeros(X_train1.shape[0])
for era in train_train_level1['era'].unique():
    mask = train_train_level1['era'] == era
    X = X_train1[mask]
    p_tr1[mask] = mdl.predict(X)
model_name_train1 = "./preds_train1/xgb_ranker_era.pkl.z"
jb.dump(p_tr1, model_name_train1)

metric = spearmanr(y_train1.values,p_tr1).correlation
    
p_vl1 = np.zeros(X_val1.shape[0])
for era in train_train_level1['era'].unique():
    mask = train_valid_level1['era'] == era
    X = X_val1[mask]
    p_vl1[mask] = mdl.predict(X)
model_name_val1 = "./preds_val1/xgb_ranker_era.pkl.z"
jb.dump(p_vl1, model_name_val1)
    
p_ts = np.zeros(X_test.shape[0])
for era in test['era'].unique():
    mask = test['era'] == era
    X = X_test[mask]
    p_ts[mask] = mdl.predict(X)
model_name_test = "./preds_test/xgb_ranker_era.pkl.z"
jb.dump(p_ts, model_name_test)
    
print(metric)

0.03775971362868848
CPU times: user 1min 28s, sys: 4.36 s, total: 1min 33s
Wall time: 21.7 s


# Random Forest

In [49]:
%%time
from xgboost import XGBRegressor, XGBRFRegressor

#https://xgboost.readthedocs.io/en/latest/tutorials/rf.html#
#subsample = 0.5, paper ~= bagging
for depth in [5, 10, 20, 50, None]:
    mdl = XGBRegressor(max_depth=depth, learning_rate=1., n_estimators=1, num_parallel_tree=1000, colsample_bytree=0.8,
                       colsample_bynode=0.8, subsample=0.5, tree_method='gpu_hist', gpu_id=0, random_state=0)
    #mdl = XGBRFRegressor(max_depth=depth, random_state=0, tree_method='gpu_hist', gpu_id=0)
    mdl.fit(X_train0, y_train0)

    p = mdl.predict(X_train1)
    model_name_train1 = "./preds_train1/xgb_rf_{}.pkl.z".format(depth)
    jb.dump(p, model_name_train1)

    metric = spearmanr(y_train1.values,p).correlation

    p = mdl.predict(X_val1)
    model_name_val1 = "./preds_val1/xgb_rf_{}.pkl.z".format(depth) 
    jb.dump(p, model_name_val1)

    p = mdl.predict(X_test)
    model_name_test = "./preds_test/xgb_rf_{}.pkl.z".format(depth)
    jb.dump(p, model_name_test)

    print(depth, metric)

5 0.004443361476974318
10 0.006305774752153184
20 0.0039169566271514454
50 0.005050604274583355
None 0.007227459512613501
CPU times: user 1min 29s, sys: 13.3 s, total: 1min 43s
Wall time: 58 s


In [58]:
mdl = XGBRFRegressor(random_state=0, tree_method='gpu_hist', gpu_id=0, n_estimators=5000, min_child_weight=10)
mdl.fit(X_train0, y_train0)

p = mdl.predict(X_train1)
model_name_train1 = "./preds_train1/xgb_rf2.pkl.z"
jb.dump(p, model_name_train1)

metric = spearmanr(y_train1.values,p).correlation

p = mdl.predict(X_val1)
model_name_val1 = "./preds_val1/xgb_rf2.pkl.z"
jb.dump(p, model_name_val1)

p = mdl.predict(X_test)
model_name_test = "./preds_test/xgb_rf2.pkl.z"
jb.dump(p, model_name_test)

print(metric)

0.005045930483233033


# Boosted Random Forest

In [63]:
%%time
from xgboost import XGBRegressor


for trees in [2, 5, 10]:
    mdl = XGBRegressor(max_depth=None, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1, subsample=1.,
                       num_parallel_tree=trees, tree_method='gpu_hist', gpu_id=0)
    mdl.fit(X_train0, y_train0)

    p = mdl.predict(X_train1)
    model_name_train1 = "./preds_train1/xgb_boosted_rf_{}.pkl.z".format(trees)
    jb.dump(p, model_name_train1)

    metric = spearmanr(y_train1.values,p).correlation

    p = mdl.predict(X_val1)
    model_name_val1 = "./preds_val1/xgb_boosted_rf_{}.pkl.z".format(trees)
    jb.dump(p, model_name_val1)

    p = mdl.predict(X_test)
    model_name_test = "./preds_test/xgb_boosted_rf_{}.pkl.z".format(trees)
    jb.dump(p, model_name_test)

    print(trees, metric)
    
#detalhe -> olhe quantas ideias testadas para conseguir, enfim, um modelo individual melhor...
# max depth...? subsample?

2 0.039648463954533336
5 0.040592441024544006
10 0.04036255365281287
CPU times: user 51min 14s, sys: 42.1 s, total: 51min 56s
Wall time: 10min 4s


In [64]:
%%time
from xgboost import XGBRegressor


for trees in [2, 5, 10]:
    mdl = XGBRegressor(max_depth=None, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1, subsample=0.5,
                       num_parallel_tree=trees, tree_method='gpu_hist', gpu_id=0)
    mdl.fit(X_train0, y_train0)

    p = mdl.predict(X_train1)
    model_name_train1 = "./preds_train1/xgb_boosted_rf2_{}.pkl.z".format(trees)
    jb.dump(p, model_name_train1)

    metric = spearmanr(y_train1.values,p).correlation

    p = mdl.predict(X_val1)
    model_name_val1 = "./preds_val1/xgb_boosted_rf2_{}.pkl.z".format(trees)
    jb.dump(p, model_name_val1)

    p = mdl.predict(X_test)
    model_name_test = "./preds_test/xgb_boosted_rf2_{}.pkl.z".format(trees)
    jb.dump(p, model_name_test)

    print(trees, metric)

2 0.03750547446034514
5 0.036692029435361304
10 0.03314805392769067
CPU times: user 44min 59s, sys: 41.3 s, total: 45min 40s
Wall time: 9min 11s
